# <font color = "red"> Utilizando a Arquitetura LTSM para prever o próximo valor da Taxa Selic

In [154]:
import pandas as pd
import numpy as np
import tensorflow as tf # modelo de deep learn
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler # normalizar os dados
from sklearn.metrics import mean_squared_error # calcular o erro
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [3]:
# Carregamento dos dados
dados = pd.read_csv("C:/Projetos Pessoais/DataScience/Previsao_Juros_BACEN_IA/data/dados_scraping")

In [4]:
dados

,Data,Taxa_Selic
0,06/11/2024,"11,25"
1,18/09/2024,"10,75"
2,31/07/2024,"10,50"
3,19/06/2024,"10,50"
4,08/05/2024,"10,50"
...,...,...
269,23/10/1996,"1,78"
270,23/09/1996,"1,82"
271,21/08/1996,"1,88"
272,30/07/1996,"1,90"


Invertendo a ordem do DataFrame, para facilitar o treinamento do modelo

In [5]:
dados_selic = dados.iloc[::-1].reset_index(drop=True)

# Exibindo o DataFrame invertido
dados_selic

,Data,Taxa_Selic
0,26/06/1996,"1,90"
1,30/07/1996,"1,90"
2,21/08/1996,"1,88"
3,23/09/1996,"1,82"
4,23/10/1996,"1,78"
...,...,...
269,08/05/2024,"10,50"
270,19/06/2024,"10,50"
271,31/07/2024,"10,50"
272,18/09/2024,"10,75"


In [6]:
dados_selic = dados_selic[48:]

In [7]:
dados_selic

,Data,Taxa_Selic
48,19/01/2000,"19,00"
49,16/02/2000,"19,00"
50,22/03/2000,"19,00"
51,28/03/2000,"18,50"
52,19/04/2000,"18,50"
...,...,...
269,08/05/2024,"10,50"
270,19/06/2024,"10,50"
271,31/07/2024,"10,50"
272,18/09/2024,"10,75"


In [169]:
import plotly.express as px

# Converter a coluna 'Data' para datetime
dados_selic['Data'] = pd.to_datetime(dados_selic['Data'], format='%d/%m/%Y')

# Criando o gráfico com Plotly
fig = px.line(
    dados_selic,
    x='Data',
    y='Taxa_Selic',
    title="Taxa Selic ao Longo do Tempo (a partir do índice 200)",
    labels={'Data': 'Data', 'Taxa_Selic': 'Taxa Selic (%)'},
    markers=True
)

# Personalizando o layout
fig.update_traces(line=dict(color='rgb(237, 96, 68)'))  # Aplica a cor personalizada na linha

fig.update_layout(
    xaxis=dict(tickformat="%Y", color = "gray"),  # Rotacionando as labels do eixo X
    yaxis=dict(title="Taxa Selic (%)", color = "gray"),
    title=dict(text="Variação da Taxa Selic (a partir de 2000)", font=dict(size=14)),
    xaxis_showgrid=True,
    yaxis_showgrid=True,
)

# Exibindo o gráfico
fig.show()

C:\Users\alyss\AppData\Local\Temp\ipykernel_5024\4018914372.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
dados.columns

Index(['Data', 'Taxa_Selic'], dtype='object')

In [10]:
dados.isna().sum()

Data          0
Taxa_Selic    0
dtype: int64

 - Alterando o tipo da taxa selic

In [11]:
dados_selic["Taxa_Selic"] = dados_selic["Taxa_Selic"].replace(",", ".", regex=True)

C:\Users\alyss\AppData\Local\Temp\ipykernel_5024\1838814633.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_selic["Taxa_Selic"] = dados_selic["Taxa_Selic"].replace(",", ".", regex=True)


In [12]:
# Alterando os tipos das variaveis
dados_selic['Taxa_Selic'] = dados_selic['Taxa_Selic'].astype(float)

C:\Users\alyss\AppData\Local\Temp\ipykernel_5024\1782901934.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_selic['Taxa_Selic'] = dados_selic['Taxa_Selic'].astype(float)


In [13]:
dados_selic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 48 to 273
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Data        226 non-null    object 
 1   Taxa_Selic  226 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.7+ KB


In [14]:
# Fazer plotagens da inflação aqui
dados_selic_plot = dados_selic.copy()

 - Iniciando Pré Processamento dos dados

In [15]:
# Transformando a taxa selic em Matriz

selic = dados_selic['Taxa_Selic'].values.reshape(-1, 1)

In [16]:
# Função para criar o dataset para o modelo LSTM com look_back
# O parâmetro look back define o numero de passos no tempo que o modelo usará para fazer cada previsõa
# a ideia de "veja os ultimos 3 dados e preveja o 4"

def func_cria_dataset(data, look_back = 1):
   
   # Inicializa duas listas vazias, X e Y, que conterão as sequências de entrada e saída
   X, Y = [], []
   # Itera pelo conjunto de dados menos o valor de look_back, Isso é feiro para evitar índices fora dos limites do array
   for i in range(len(data) - look_back):      
      # Coleta uma sequência de dados de tamanho look_back começando no índice i
      a = data[i:(i + look_back), 0]
      # Adiciona a sequência à lista X
      X.append(a)
      # Adiciona o valor imediatamente após a sequência de look_back à lista Y. Esse valor será o target.
      Y.append(data[i + look_back, 0])

   # Converte X e Y para arrays numpy para compatibilidade com a maioria das bibliotecas de machine learninog
   return np.array(X), np.array(Y)

In [17]:
# Dividimos os dados em treinamento e teste (respeitando a ordem cronológica dos dados)

indice = int(len(selic) * 0.85)
dados_treino, dados_teste = selic[0:indice, :], selic[indice:len(selic), :] 

In [18]:
dados_treino[:10]

array([[19. ],
       [19. ],
       [19. ],
       [18.5],
       [18.5],
       [18.5],
       [17.5],
       [17. ],
       [16.5],
       [16.5]])

In [19]:
len(dados_treino), len(dados_teste)

(192, 34)

In [20]:
# Normaliza os dados (Requerimento para redes neurais)

scaler = MinMaxScaler(feature_range = (0, 1))

In [21]:
# Treina e aplica o sclaer emt reino e somente aplica em teste

dados_treino_norm = scaler.fit_transform(dados_treino)
dados_teste_norm = scaler.transform(dados_teste)

In [22]:
dados_teste_norm[:20]

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.03061224],
       [0.06122449],
       [0.09183673],
       [0.13265306],
       [0.17346939],
       [0.23469388],
       [0.29591837],
       [0.35714286],
       [0.39795918],
       [0.43877551],
       [0.45918367],
       [0.47959184],
       [0.47959184],
       [0.47959184],
       [0.47959184],
       [0.47959184]])

In [23]:
#  Criamos os datasets para o modelo LSTM

look_back = 3
X_treino, y_treino = func_cria_dataset(dados_treino_norm, look_back)
X_teste, y_teste = func_cria_dataset(dados_teste_norm, look_back)

In [24]:
X_treino[:5]

array([[0.69387755, 0.69387755, 0.69387755],
       [0.69387755, 0.69387755, 0.67346939],
       [0.69387755, 0.67346939, 0.67346939],
       [0.67346939, 0.67346939, 0.67346939],
       [0.67346939, 0.67346939, 0.63265306]])

In [25]:
y_treino[:9]

array([0.67346939, 0.67346939, 0.67346939, 0.63265306, 0.6122449 ,
       0.59183673, 0.59183673, 0.59183673, 0.59183673])

In [26]:
#  Reshape dos dados para [samples, time steps, features] - Requerimento para o modelo LSTM

X_treino = np.reshape(X_treino, (X_treino.shape[0], X_treino.shape[1], 1))
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))

In [27]:
X_treino[:2]

array([[[0.69387755],
        [0.69387755],
        [0.69387755]],

       [[0.69387755],
        [0.69387755],
        [0.67346939]]])

## <font color = "red"> Construção do Modelo LSTM

In [49]:
# 50 neurônios artificais

modelo = tf.keras.models.Sequential([tf.keras.layers.LSTM(50, input_shape = (look_back, 1)),
                                     tf.keras.layers.Dense(1)])

In [50]:
# Compila o modelo
# Nesta etapa, aplica o algorítmo de otimização que aplica o back propagation, no caso foi o "adam"
# função de erro 
modelo.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [51]:
# Treinamento do modelo
# Batch_size = alimenta 1 registro por vez
# verbose mostra na tela o treinmaneot em andamento

modelo.fit(X_treino, y_treino, epochs = 50, batch_size = 1, verbose = 1)

Epoch 1/50
189/189 [==============================] - 2s 2ms/step - loss: 0.0352
Epoch 2/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 3/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 4/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0030
Epoch 5/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 6/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0030
Epoch 7/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 8/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 9/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 10/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 11/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 12/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0027
Epoch 13/50
189/189 [====

In [52]:
# Fazendo previsões com o modelo

previsao_treino = modelo.predict(X_treino)
previsao_teste = modelo.predict(X_teste)

1/1 [==============================] - 0s 21ms/step


In [155]:
# Modelo 2
modelo2 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(look_back, 1)),
    tf.keras.layers.Dense(1)
])

modelo2.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')

modelo2.fit(X_treino, y_treino, epochs = 50, batch_size = 1, verbose = 1)

Epoch 1/50
189/189 [==============================] - 2s 2ms/step - loss: 0.0246
Epoch 2/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 3/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0027
Epoch 4/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0026
Epoch 5/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0026
Epoch 6/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 7/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 8/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 9/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 10/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0021
Epoch 11/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 12/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 13/50
189/189 [====

In [157]:
previsao_treino2 = modelo2.predict(X_treino)
previsao_teste2 = modelo2.predict(X_teste)

1/1 [==============================] - 0s 55ms/step


In [158]:
# Transformar de volta para a escala orignial e calcular o erro do modelo

previsao_treino2 = scaler.inverse_transform(previsao_treino2)
y_treino_rescaled2 = scaler.inverse_transform([y_treino])
previsao_teste2 = scaler.inverse_transform(previsao_teste2)
y_teste_rescaled2 = scaler.inverse_transform([y_teste])

In [159]:
# Calcula o RMSE

train_score2 = np.sqrt(mean_squared_error(y_treino_rescaled2[0],
                                         previsao_treino2[:,0]))
print(f"\nRMSE em treino: {train_score2:.2f}\n")

teste_score2 = np.sqrt(mean_squared_error(y_teste_rescaled2[0],
                                         previsao_teste2[:,0]))
print(f"\nRMSE em teste: {teste_score2:.2f}\n")


RMSE em treino: 0.49


RMSE em teste: 0.63



## CALCULANDO ERRO do modelo 1

In [53]:
# Transformar de volta para a escala orignial e calcular o erro do modelo

previsao_treino = scaler.inverse_transform(previsao_treino)
y_treino_rescaled = scaler.inverse_transform([y_treino])
previsao_teste = scaler.inverse_transform(previsao_teste)
y_teste_rescaled = scaler.inverse_transform([y_teste])

In [54]:
y_teste_rescaled

array([[ 2.  ,  2.75,  3.5 ,  4.25,  5.25,  6.25,  7.75,  9.25, 10.75,
        11.75, 12.75, 13.25, 13.75, 13.75, 13.75, 13.75, 13.75, 13.75,
        13.75, 13.75, 13.25, 12.75, 12.25, 11.75, 11.25, 10.75, 10.5 ,
        10.5 , 10.5 , 10.75, 11.25]])

In [55]:
# Calcula o RMSE

train_score = np.sqrt(mean_squared_error(y_treino_rescaled[0],
                                         previsao_treino[:,0]))
print(f"\nRMSE em treino: {train_score:.2f}\n")

teste_score = np.sqrt(mean_squared_error(y_teste_rescaled[0],
                                         previsao_teste[:,0]))
print(f"\nRMSE em teste: {teste_score:.2f}\n")


RMSE em treino: 0.50


RMSE em teste: 0.54



In [56]:
last_data = dados_teste_norm[-look_back:]
last_data = np.reshape(last_data, (1, look_back, 1))

# Lista
lista_previsoes = []

# Loop de Previsão para prever 2 anos (2024 e 2025)
for _ in range(2):  

    # Previsão com o modelo (usamos os dados normalizados)
    prediction = modelo.predict(last_data)

    # Adiciona a previsão à lista de previsões
    lista_previsoes.append(prediction[0, 0])

    # Atualiza os dados para incluir a nova previsão e remover o valor mais antigo
    last_data = np.roll(last_data, shift = -1)
    last_data[0, look_back - 1, 0] = prediction

# Transformar de volta para a escala original
lista_previsoes_rescaled = scaler.inverse_transform(np.array(lista_previsoes).reshape(-1, 1))

print(f"\nPrevisão da Taxa de Juros Para 2024: {lista_previsoes_rescaled[0, 0]:.2f}")
print(f"Previsão da Inflação Para 2025: {lista_previsoes_rescaled[1, 0]:.2f}")

print("\nLab Concluído. Obrigado, DSA!")


1/1 [==============================] - 0s 20ms/step

Previsão da Taxa de Juros Para 2024: 11.62
Previsão da Inflação Para 2025: 12.03

Lab Concluído. Obrigado, DSA!


In [57]:
def arredondar_decimal(valor):
    # Multiplicamos por 20, arredondamos para o inteiro mais próximo e depois dividimos por 20
    return math.floor(valor * 20) / 20
    


In [58]:
dict_previsao = {
   'Data': ["11/12/2024", "29/01/2025"],
   'Taxa_Selic': [arredondar_decimal(round(lista_previsoes_rescaled[0, 0],2)), arredondar_decimal(round(lista_previsoes_rescaled[1, 0],2))]
}

previsoes = pd.DataFrame(dict_previsao)

In [59]:
previsoes

,Data,Taxa_Selic
0,11/12/2024,11.6
1,29/01/2025,12.0


In [75]:
import plotly.express as px

# Convertendo a coluna "Data" para datetime
dados_selic["Data"] = pd.to_datetime(dados_selic["Data"], format="%d/%m/%Y")
previsoes["Data"] = pd.to_datetime(previsoes["Data"], format="%d/%m/%Y")

# Adicionando a coluna "Origem"
dados_selic["Origem"] = "Histórico"
previsoes["Origem"] = "Previsão"

# Concatenando os DataFrames
df_unido = pd.concat([dados_selic, previsoes], ignore_index=True)

# Filtrando dados a partir de 2010
df_filtrado = df_unido[df_unido["Data"] >= "2020-01-01"]


C:\Users\alyss\AppData\Local\Temp\ipykernel_5024\3599250906.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\alyss\AppData\Local\Temp\ipykernel_5024\3599250906.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [153]:
# # Plot com a paleta ajustada
fig = px.line(
    df_filtrado,
    x="Data",
    y="Taxa_Selic",
    color="Origem",
    title="Taxa Selic (2010 em diante)",
    labels={"Data": "Data (Ano)", "Taxa_Selic": "Taxa Selic (%)"},
    color_discrete_map=color_discrete_map,
    markers=True
)

# Adicionando os valores como texto nos pontos de previsão
for i, row in df_filtrado[df_filtrado["Origem"] == "Previsão"].iterrows():
    fig.add_annotation(
        x=row["Data"],
        y=row["Taxa_Selic"],
        text=f"{row['Taxa_Selic']}%",  # Exibe o valor formatado
        showarrow=True,
        arrowhead=5,
        ax=10,
        ay=30,
        font=dict(color="rgb(164, 0, 0)", size=12),
        textangle=-40,
        arrowcolor="rgb(164, 0, 0)"
    )

fig.update_layout(
    xaxis=dict(
        tickformat="%Y",
        title=dict(text="Data (Ano)", font=dict(color="gray")),  # Cor do título do eixo X
        tickfont=dict(color="lightgray", size=13)  # Cor dos ticks do eixo X
    ),
    yaxis=dict(
        title=dict(text="Taxa Selic (%)", font=dict(color="gray")),  # Cor do título do eixo Y
        tickfont=dict(color="lightgray", size=13)  # Cor dos ticks do eixo Y
    ),
    title=dict(
        text="Taxa Selic (2020 em diante)",  # Texto do título
        font=dict(color="gray")  # Título em negrito
    )
)


# Ajustando o layout
fig.update_layout(xaxis=dict(tickformat="%Y"))
fig.show()